In [1]:
import os
import re
import ast
import numpy as np
import pandas as pd
from collections import defaultdict


def parse_ranpac_log(log_file_path):
    """Parse a RanPAC log file to extract metrics."""
    with open(log_file_path, 'r') as f:
        lines = f.readlines()
    
    result = {
        "model_name": "",
        "convnet_type": "",
        "exp_name": "",
        "dataset": "",
        "seed": "",
        "final_accuracy": 0.0,
        "average_accuracy": 0.0,
        "forgetting": 0.0,
    }
    
    # Parse header information
    for line in lines:
        if "model_name:" in line:
            result["model_name"] = line.split("model_name:")[1].strip()
        elif "convnet_type:" in line:
            result["convnet_type"] = line.split("convnet_type:")[1].strip()
        elif "exp_name:" in line:
            result["exp_name"] = line.split("exp_name:")[1].strip()
        elif "dataset:" in line:
            result["dataset"] = line.split("dataset:")[1].strip()
        elif "seed:" in line:
            result["seed"] = line.split("seed:")[1].strip()
    
    # Extract filename seed if not found in log
    if not result["seed"]:
        filename = os.path.basename(log_file_path)
        seed_match = re.search(r'(\d{4})_', filename)
        if seed_match:
            result["seed"] = seed_match.group(1)
    
    # Extract final Top1 curve for final_accuracy and average_accuracy
    top1_curves = []
    for line in lines:
        if "Top1 curve:" in line:
            # Extract the list from the line
            curve_str = line.split("Top1 curve:")[1].strip()
            try:
                curve_values = ast.literal_eval(curve_str)
                top1_curves.append(curve_values)
            except:
                continue
    
    if top1_curves:
        final_curve = top1_curves[-1]  # Last Top1 curve
        result["final_accuracy"] = final_curve[-1]  # Last value in the curve
        result["average_accuracy"] = np.mean(final_curve)  # Mean of all values
    
    # Extract Group Accuracies for forgetting calculation
    group_accuracies_history = defaultdict(list)
    
    for line in lines:
        if "Group Accuracies after this task:" in line:
            # Extract the dictionary from the line
            acc_str = line.split("Group Accuracies after this task:")[1].strip()
            try:
                # Parse the dictionary-like string
                acc_dict = ast.literal_eval(acc_str)
                for group, accuracy in acc_dict.items():
                    group_accuracies_history[group].append(accuracy)
            except:
                continue
    
    # Calculate forgetting: last - max for each group, then average
    forgetting_values = []
    for group, accuracies in group_accuracies_history.items():
        if len(accuracies) > 1:  # Need at least 2 values to calculate forgetting
            max_acc = max(accuracies)
            last_acc = accuracies[-1]
            forgetting = max_acc - last_acc  # Forgetting = max - last
            forgetting_values.append(forgetting)
    
    if forgetting_values:
        result["forgetting"] = np.mean(forgetting_values)
    
    return result


def parse_logs_folder(logs_folder):
    """Parse all log files in the logs folder."""
    all_results = []
    
    # Find all .log files recursively
    for root, dirs, files in os.walk(logs_folder):
        for file in files:
            if file.endswith('.log'):
                log_path = os.path.join(root, file)
                print(f"Processing: {log_path}")
                try:
                    result = parse_ranpac_log(log_path)
                    # Add path information for debugging
                    result["log_path"] = log_path
                    
                    # Extract exp_name from path if not found in log
                    if not result["exp_name"]:
                        path_parts = log_path.split(os.sep)
                        if len(path_parts) >= 2:
                            result["exp_name"] = path_parts[-2]  # Parent directory name
                    
                    all_results.append(result)
                except Exception as e:
                    print(f"Error processing {log_path}: {e}")
                    continue
    
    return all_results


def generate_csv_results(logs_folder, output_prefix="ranpac_results"):
    """Generate CSV files with results and statistics."""
    all_results = parse_logs_folder(logs_folder)
    
    if not all_results:
        print("No valid results found!")
        return
    
    # Create DataFrame
    df = pd.DataFrame(all_results)
    
    # Rearrange columns
    columns = ["model_name", "convnet_type", "exp_name", "dataset", "seed", 
               "final_accuracy", "average_accuracy", "forgetting", "log_path"]
    df = df[columns]
    
    # Sort by relevant columns
    df = df.sort_values(by=["model_name", "dataset", "convnet_type", "exp_name", "seed"])
    
    # Save raw results
    df.to_csv(f"{output_prefix}_raw.csv", index=False)
    print(f"Saved raw results to {output_prefix}_raw.csv")
    
    # Group by experiment settings and calculate mean/std
    groupby_cols = ["model_name", "convnet_type", "exp_name", "dataset"]
    
    # Check which experiments have complete seeds
    grouped = df.groupby(groupby_cols)
    complete_experiments = []
    incomplete_experiments = []
    
    for name, group in grouped:
        seeds = sorted(group["seed"].astype(str).tolist())
        if len(seeds) >= 3:  # Assuming we want at least 3 seeds
            complete_experiments.append((name, group))
        else:
            incomplete_experiments.append((name, group, seeds))
    
    print(f"\nFound {len(complete_experiments)} complete experiments")
    print(f"Found {len(incomplete_experiments)} incomplete experiments")
    
    if incomplete_experiments:
        print("\nIncomplete experiments:")
        for name, group, seeds in incomplete_experiments:
            print(f"  {name}: seeds {seeds}")
    
    # Calculate mean and std for complete experiments
    if complete_experiments:
        stats_data = []
        
        for name, group in complete_experiments:
            stats = {
                "model_name": name[0],
                "convnet_type": name[1], 
                "exp_name": name[2],
                "dataset": name[3],
                "num_seeds": len(group),
                "seeds": ",".join(sorted(group["seed"].astype(str))),
                "final_accuracy_mean": group["final_accuracy"].mean(),
                "final_accuracy_std": group["final_accuracy"].std(),
                "average_accuracy_mean": group["average_accuracy"].mean(),
                "average_accuracy_std": group["average_accuracy"].std(),
                "forgetting_mean": group["forgetting"].mean(),
                "forgetting_std": group["forgetting"].std(),
            }
            stats_data.append(stats)
        
        stats_df = pd.DataFrame(stats_data)
        stats_df = stats_df.sort_values(by=["model_name", "dataset", "convnet_type", "exp_name"])
        stats_df.to_csv(f"{output_prefix}_stats.csv", index=False)
        print(f"Saved statistics to {output_prefix}_stats.csv")
        
        # Display summary
        print("\nSummary of results:")
        print(stats_df.to_string(index=False))
    
    return df


# Main execution
logs_folder = "../logs"
df = generate_csv_results(logs_folder)


Processing: ../logs/adapter/cifar224/0/10/768_1/1993_pretrained_vit_b16_224_in21k_adapter.log
Processing: ../logs/adapter/cifar224/0/10/768_1/1995_pretrained_vit_b16_224_in21k_adapter.log
Processing: ../logs/adapter/cifar224/0/10/768_1/1994_pretrained_vit_b16_224_in21k_adapter.log
Processing: ../logs/adapter/cifar224/0/10/default/1993_pretrained_vit_b16_224_in21k_adapter.log
Processing: ../logs/adapter/cifar224/0/10/default/1995_pretrained_vit_b16_224_in21k_adapter.log
Processing: ../logs/adapter/cifar224/0/10/default/1994_pretrained_vit_b16_224_in21k_adapter.log
Processing: ../logs/adapter/cifar224/0/10/2000_1/1993_pretrained_vit_b16_224_in21k_adapter.log
Processing: ../logs/adapter/cifar224/0/10/2000_1/1995_pretrained_vit_b16_224_in21k_adapter.log
Processing: ../logs/adapter/cifar224/0/10/2000_1/1994_pretrained_vit_b16_224_in21k_adapter.log
Processing: ../logs/adapter/cifar224/0/10/mean_2000_5/1993_pretrained_vit_b16_224_in21k_adapter.log
Processing: ../logs/adapter/cifar224/0/10/mea